In [10]:
import polars as pl
df = pl.read_excel("C:/Users/Wilcoxon/Desktop/etf.xlsx")

In [11]:
df = df.pivot(on = ["INDUSTRIESNAME_1"],index="EndDate",values="asset").fill_null(0)

In [12]:
df.write_excel("d:/etf.xlsx")

In [13]:
df

EndDate,股票型,QDII,债券型,商品型
date,f64,f64,f64,f64
2005-02-28,47.90797,0.0,0.0,0.0
2005-03-31,52.05044,0.0,0.0,0.0
2005-04-29,62.95608,0.0,0.0,0.0
2005-05-31,65.872,0.0,0.0,0.0
2005-06-30,67.65296,0.0,0.0,0.0
…,…,…,…,…
2024-07-31,20573.239862,2826.202482,1281.879628,546.670729
2024-08-30,21116.221006,2869.148995,1308.202401,536.321679
2024-09-30,27967.128105,3326.640615,1373.028403,569.018392


In [23]:
from quant_utils.db_conn import DB_CONN_JJTG_DATA
query_sql = """
SELECT
  a.EndDate,
  c.INDUSTRIESNAME_1,
  sum(a.FloatShares) / 100000000 AS shares, sum(a.FloatShares * d.UNIT_NAV) / 100000000 AS asset 
FROM
  jy_local.mf_shareschange a
  JOIN chentiancheng.fund_info b ON a.InnerCode = b.SECURITY_ID 
  AND b.ETF_LOF = 'ETF' 
  AND b.ESTABLISH_DATE <= a.EndDate
  JOIN chentiancheng.fund_type_jy c ON c.TICKER_SYMBOL = b.TICKER_SYMBOL 
  AND c.INDUSTRIESNAME_1 IN ('QDII', '商品型') 
  AND c.CUR_SIGN = 1
  JOIN chentiancheng.fund_adj_nav d ON d.TICKER_SYMBOL = b.TICKER_SYMBOL 
  AND d.END_DATE = a.EndDate
  JOIN md_tradingdaynew e ON e.TRADE_DT = a.EndDate 
  AND e.SECU_MARKET = 83 
  AND e.IF_MONTH_END = 1 
WHERE
  1 = 1 
  AND (a.StatPeriod = 996 OR (a.StatPeriod = 993 AND a.InfoSource LIKE '%上市%' AND a.InfoSource LIKE '%公告%')) 
GROUP BY
  a.EndDate,
  c.INDUSTRIESNAME_1 
ORDER BY
  a.EndDate,
  c.INDUSTRIESNAME_1;
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [24]:
df = df.pivot_table(index="EndDate",columns="INDUSTRIESNAME_1",values="shares").fillna(0)

C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_25168\891879739.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.pivot_table(index="EndDate",columns="INDUSTRIESNAME_1",values="shares").fillna(0)


In [25]:
df

INDUSTRIESNAME_1,QDII,商品型
EndDate,,
2012-10-31,22.105592,0.000000
2012-11-30,10.365592,0.000000
2012-12-31,8.115592,0.000000
2013-01-31,5.955592,0.000000
2013-02-28,5.225592,0.000000
...,...,...
2024-07-31,3922.142173,108.722687
2024-08-30,3927.434540,105.811687
2024-09-30,3878.511540,110.011687


In [26]:
df.to_excel("d:/linglei.xlsx")